In [1]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"
model_name="../model/------llm-models-hf-0503llama-_inst_gousei_lr_5e-6"
model_name="../model/------llm-models-hf-2jaA_inst_gousei_lr_1e-5/checkpoint-47"
model_name="../model/------llm-models-hf-2jaA_inst_gousei_lr_5e-6"
model_name="Local-Novel-LLM-project/Vecteus-v1"
#model_name="nvidia/Llama3-ChatQA-1.5-8B"
#model_name="../model/------llm-models-hf-1code_inst_gousei_lr_5e-6"

#model_name="../X_merge/mergoo_llama_test"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )
#層の表示
#for name, param in model.named_parameters():
#    print(name)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


8


Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.55s/it]


In [2]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [3]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=300, 
              repetition_penalty=1.2,
              temperature=0.6,
              )


In [5]:
question="こんにちは"
question="元気ですか?"
#question="いい天気ですね"
#question="明日の天気は?"
#uestion="計算してください 1+1はいくつですか｡"
question="calculate the folowing quation: 1+1"
#question=": 1+1はいくつですか"
#question="いい天気ですね"
#question="フィボナッチ数列を生成するpythonのコードを作成してください"
question="Please create a python code that generates the Fibonacci sequence."
#question="1+1はいくつですか"
question="ドラえもんの友達はだれですか?"
#question="レイリー散乱とはなんですか"
question="hello!"
#question="今の天気は晴れです｡この結果をjsonで出力してください｡ "

question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
inp=f"{question_template}{question}<SEP>応答<SEP>"

inp="""次の質疑の回答を1-5点で評価してください
Q
以下の中学受験国語/演習ので、「ほっさ」という表現を含む文章を日本語で作成してください。 質問：「ほっさ」という表現が使われる文章を日本語で作成してください。

A
「今朝、山登りに行ったのですが、そこは綺麗な景色が見事でした。少し困難な所もありましたが、一歩先を目指して、頑張りました。やっと山の頂上に着いた時、「ほっさ」と息を胸から放ちました。」
評価:
"""

#inp="this is a pen"
#question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。指示"
#inp=f"{question_template}{question}応答"
#inp=f"{question}\n応答:"
print(pipe(inp)[0]["generated_text"][len(inp):])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---
正解発表！
Aの文章が正解です！
「ほっさ」とは、達成感や開放感を表す方言的な表現で、主に九州地方で用いられることが多いようですね。他の地域では通じない場合もあるので注意が必要ですが、心情描写に効果的な言葉です。
